# Notebook 4: Attribute Node Enrichment

This notebook processes the LLM responses for important entities' attributes.  

Each attribute is inserted into the **G1 - primary graph** as an **Attribute (A) node** and linked to its corresponding entity node.  

The resulting graph is saved as **G2 - attribute-enriched graph**.


In [1]:
import os
dir_path = os.getcwd()
print("The directory of this script is:", dir_path)
root_path = os.path.dirname(dir_path)
print("The root directory is:", root_path)

The directory of this script is: c:\Users\HP\Desktop\Projects\NodeRAG\graphs
The root directory is: c:\Users\HP\Desktop\Projects\NodeRAG


In [2]:
import sys
sys.path.append(root_path)
from graphs.Node import Node

In [3]:
import pickle
with open(f"{root_path}/graphs/data/graphs/G1_medical_primary_graph.pkl", "rb") as f:
    medical_g1 = pickle.load(f)
medical_g1

{'medical-0-S-0': <graphs.Node.Node at 0x2574a8da5f0>,
 'medical-0-S-0-R-0': <graphs.Node.Node at 0x2574ae54ca0>,
 'medical-0-S-0-R-1': <graphs.Node.Node at 0x2574ae55480>,
 'medical-0-S-0-R-2': <graphs.Node.Node at 0x2574ae552d0>,
 'medical-0-S-1': <graphs.Node.Node at 0x2574ae55240>,
 'medical-0-S-1-R-0': <graphs.Node.Node at 0x2574ae54c70>,
 'medical-0-S-1-R-1': <graphs.Node.Node at 0x2574ae54a30>,
 'medical-0-S-1-R-2': <graphs.Node.Node at 0x2574ae55390>,
 'medical-0-S-1-R-3': <graphs.Node.Node at 0x2574ae555a0>,
 'medical-0-S-1-R-4': <graphs.Node.Node at 0x2574ae556f0>,
 'medical-0-S-1-R-5': <graphs.Node.Node at 0x2574ae554e0>,
 'medical-0-S-1-R-6': <graphs.Node.Node at 0x2574ae546d0>,
 'medical-0-S-1-R-7': <graphs.Node.Node at 0x2574ae544c0>,
 'medical-0-S-2': <graphs.Node.Node at 0x2574ae55600>,
 'medical-0-S-2-R-0': <graphs.Node.Node at 0x2574ae55510>,
 'medical-0-S-2-R-1': <graphs.Node.Node at 0x2574ae54d60>,
 'medical-0-S-2-R-2': <graphs.Node.Node at 0x2574ae558a0>,
 'medical

In [4]:
import pandas as pd
medical_attributes = pd.read_parquet("data/nodes/attribute/medical_attributes.parquet")
medical_attributes

,index,attribute
0,medical-N-0,"Basal Cell Skin Cancer, often called Basal Cel..."
1,medical-N-3,A vast land of remarkable contrasts and intric...
2,medical-N-4,"Surgery is a primary intervention, a physical ..."
3,medical-N-5,"At the heart of our body's shield, the skin, l..."
4,medical-N-6,"The epidermis is the skin's outermost defense,..."
...,...,...
681,medical-N-6571,A survivor of anal cancer walks a path forged ...
682,medical-N-6576,"The Digital Rectal Exam, or DRE, is a crucial ..."
683,medical-N-6587,Intensity-Modulated Radiation Therapy (IMRT) s...
684,medical-N-6686,"Abdominoperineal Resection, or APR, is a signi..."


In [5]:
from tqdm import tqdm
for idx, row in tqdm(medical_attributes.iterrows()):
    entity_id = row['index']
    entity_node = medical_g1[entity_id]
    attribute = row['attribute']
    attribute_node = Node(id = f"{entity_id}-A-0", node_type="A", source="", content=attribute)
    entity_node.link(attribute_node)
    attribute_node.link(entity_node)
    medical_g1[attribute_node.id] = attribute_node

    if idx % 100 == 0:
        attribute_node.print()
        print("-"*40)

686it [00:00, 14712.42it/s]

ID: medical-N-0-A-0
Type: A
Source: 
Edges: {'medical-N-0': 1}
Content: Basal Cell Skin Cancer, often called Basal Cell Carcinoma (BCC), is the most common form of its kind, an unwelcome visitor that affects millions each year. It's a condition born from our skin's basal cells, those fundamental cells in the epidermis, deciding to grow beyond control, often under the relentless gaze of the sun. These cancers commonly choose exposed territories like the face, head, neck, arms, and legs as their staging ground, though they can appear anywhere. While anyone can develop it, those with lighter skin, hair, and eyes are particularly vulnerable, their skin offering less natural defense against the damaging UV radiation from the sun or tanning beds. For those with fair complexions, BCC can manifest as subtle yet concerning changes: flat, scar-like patches, itchy red areas, small shiny bumps with unusual hues, or persistent sores that bleed. In darker skin tones, it might present as a glossy bla

In [6]:
print(len(medical_g1))
with open(f"{root_path}/graphs/data/graphs/G2_medical_attribute_enriched_graph.pkl", "wb") as f:
    pickle.dump(medical_g1, f)

32127
